<center><a href="https://www.nvidia.cn/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

# <font color="#76b900">**5:** 适用于多模态推理的 Transformer</font>

之前，我们深入研究了使用编码器和解码器组件的模型，它们展示出了处理自然语言的强大能力。这些模型通过自回归解码将自然语言指令高效的转成结构化输出，同时用交叉注意力来引导生成。虽然这种编码器-解码器架构很符合直觉，也在我们的实验中表现很好，但或许会有这样的疑问：**将语境和生成过程分离是否总是最佳的选择？**

我们很快就会了解到，对于先进的模型架构来说，编码器-解码器的组合可能不是必须的。不过在开始之前，我们应该先看看编码器-解码器架构真正擅长的领域：**多模态推理**。

## 5.1. 什么是模态？

***“模态”***指的是具有不同基本规则和直觉的人类能够感知的数据类型。

在之前的学习中，您了解过卷积作为一种稀疏推理策略，有助于将空间局部性纳入模型的推理过程中。我们知道这对将图像转换为特征集非常有用，因此我们把这个架构叫作“特征提取器”。通常来讲它们对密集推理也很有用，尽管由于维数灾难（curse of dimensionality）可能会变得效率低下。但只要时间够用，它们肯定能推理的不错，对吧？

所有的数据都是这样：每个模态都在比特层面有其内在的关系模式，它们需要用相应的架构来处理。我们一直在探讨 transformer 组件凭借其 token 级的密集层和轻量的注意力接口成为出色的语言推理工具，但要记住，这只是因为**语言是有序序列**。注意力机制就是擅长处理序列化的数据，而这实际上可以不仅局限于单词和句子。

> <div><img src="imgs/multimodal.png" width="800"/></div>
> 
> **来源: [NVIDIA Keynote at SIGGRAPH 2023 | NVIDIA](https://www.nvidia.com/en-us/events/siggraph/)**

在这几个 notebook 中，我们主要会接触到以下几种模态：

* **自然语言：**本质上是一个有序的 token 序列，尽管在不同语境下词汇和结构会有所不同。
* **图像：**在 2D 平面上表达微观（细粒度）和宏观（全局）特征的像素集合。
* **音频：**时间序列数据，其中每个时间点都对应着传感器采集到的数据。

虽然每种模态在结构和数据上都有细微的差别，但当用于指导文本生成时，就会统一起来。在此 notebook 中，我们将从比较高的层面构建对该系统的直觉，然后上手用一用，看它们都能做什么！

## 5.2. 交叉注意力的本质

我们之前介绍过交叉注意力的概念，就是一个 $n$ 个元素的序列在 $m$ 个元素的语境的指导下，生成第 $n+1$ 个元素的过程。这种技术同时利用了静态的语境编码和生成的 token 来实现自回归文本生成。

知道了这一点，如果我们想用不同于文本的另一种数据模态作为生成的上下文会发生什么？事实证明，只要我们依然按照序列来处理数据，上述机制就不会改变。

我们在这个 notebook 中将面临的主要挑战是：
> **我们还能把什么作为序列输入来指导输出？**

之前，我们提及了与自然语言明显不同的两种模态：图像和音频。我们发现，虽然音频自然就是序列化的模式，但图像却不是。让我们继续导入一些 pipeline，仔细探究一番！

## 5.3. 用 LLM 做自动语音识别

**自动语音识别（ASR）** 是识别和推理原始音频中语音的一项任务。最常见的功能是自动转录，您看到的自动生成字幕的视频流都是调用了自动语音识别。

> <div><img src="imgs/asr-pipe.png" width="800"/></div>
> 
> **来源: [Speeding Up Development of Speech and Language Models with NVIDIA NeMo](https://www.nvidia.com/en-us/events/siggraph/)**

正如您所料，训练并让这类系统能实际使用是一项艰巨的工作，需要大量的领域知识，其中一些知识可能在[这篇 NVIDIA 技术博客](https://developer.nvidia.com/blog/essential-guide-to-automatic-speech-recognition-technology/)有提到，总体的思路其实很简单：

* 将音频分割成固定大小的时间窗口。
* 将时间窗口转成名为[**频谱图**](https://en.wikipedia.org/wiki/Spectrogram)的图片，适当缩放以适应人类感官的范围（因此经常会看到对数梅尔尺度，log-mel-scale），并中和自然噪声。
* 用神经网络来推理这些图像，获得一个值的序列。
* 加上 transformer 组件，然后像之前一样通过交叉注意力与解码器交互。

换言之，除了编码器的输入过程（进入 transofmer 之前的编码）和研究团队的一些创新之外，没有什么主要变化。这些变化或许是由于他们发现实际中这样设计效果更好，或许是他们需要针对任务做一些监督上的调整。无论哪种，架构的核心仍然没有变，系统依旧运行的很好！

### 拉取 Whisper 模型

为了展示一个最前沿的例子，我们将引入 [OpenAI 的 Whisper 模型](https://openai.com/research/whisper)，HuggingFace 提供了完整的 pipeline 支持，并有各种模型大小供选择。其架构与我们描述的非常相似，许多使模型在实际中效果很好的细节都被隐藏在训练目标、多语种数据集、两端的流程（输入/后处理）和超参数中。

> <div><img src="imgs/whisper-arch.png" width="800"/></div>
>
> **来源: [Introducing Whisper | OpenAI](https://openai.com/research/whisper)**

最热门的变体之一就是 [`whisper-large-v2`](https://huggingface.co/openai/whisper-large-v2) 模型。它有 15.5 亿参数，这对 LLM 来说其实并不太大，但它在多语言数据转录方面表现非常好。尽管我们可以放心的加载此模型，但我们还是先拉取 [`whisper-base`](https://huggingface.co/openai/whisper-base) 版本。它只有大概 7300 万个参数，不仅运行速度明显更快，还出奇的好用。因此对于不需要最佳格式和准确性的应用来说，选它准没错。

In [6]:
from transformers import pipeline

## whisper-large-v2 recommended, but slow. For only english, whisper-base is fine
## Upgrading to a larger model will lead to better hard-case results and formatting.
model_name = ["openai/whisper-base", "openai/whisper-large-v2"][0] 
pipe_asr = pipeline("automatic-speech-recognition", model=model_name)

我们将用几个经典的例子来演示：

[**HuggingFace Audio Book 的语音演示**](https://huggingface.co/openai/whisper-base)

* [`sample1.flac`](https://cdn-media.huggingface.co/speech_samples/sample1.flac)
* [`sample2.flac`](https://cdn-media.huggingface.co/speech_samples/sample2.flac)
* [`sample3.flac`](https://cdn-media.huggingface.co/speech_samples/sample3.flac)

[**OpenAI Whisper 的演示**](https://openai.com/research/whisper)

* [`w_micro-machines.wav`*（快速的说话声）*](https://cdn.openai.com/whisper/draft-20220913a/micro-machines.wav)
* [`w_multilingual.wav`*（法语语音）*](https://cdn.openai.com/whisper/draft-20220920a/multilingual.wav)
* [`w_scottish-accent.wav`*（带口音的英语）*](https://cdn.openai.com/whisper/draft-20220920a/scottish-accent.wav)
* [`w_younha.wav`*（韩语流行音乐）*](https://cdn.openai.com/whisper/draft-20220913a/younha.wav)

让我们运行 pipeline，看看它怎么执行这些任务：

In [ ]:
from glob import glob

## This process can take a bit of time per entry, so we'll just generate them one at a time.
for f in sorted(glob("audio-files/*")):
    print(f, f"Prediction: {pipe_asr(f)['text']}", '', sep='\n')

----

正如我们所看到的，这个模型在英语上表现的非常好（[`whisper-large-v2`](https://huggingface.co/openai/whisper-large-v2) 在其它语言上也表现的很好），而且仅用了大约 7300 万个参数！

In [ ]:
print(f"""whisper-base:
 - model size: {pipe_asr.model.num_parameters():,} parameters
 - memory footprint: {pipe_asr.model.get_memory_footprint()/1e9 :.03f} GB""")

如果想的话，可以调查模型看看是不是我们所熟悉的架构，确实是！请记住 `WhisperDecoderLayer` 中的第一个 `WhisperAttention` 是用于自注意力的，第二个用于交叉注意力的。

In [ ]:
pipe_asr.model

----

借助此模型，您可以尽情发挥想象力。但请记住，为验证模型是否能满足您的要求，需要做严格的测试。正如您之前所看到的，如果需要对任意语言的音频进行推理，`base` 模型可能就不适合了。如果计算资源有限或者同时有很多其它模型在运行，那么 `large-v2` 就是不合适的。

此外请记住，为了建立整体的直觉，我们没有讨论很多领域相关的细节，比如输入的处理方式、损失函数的设计、训练技巧以及 token 生成流程等等。如果想了解更多信息，我们强烈建议您阅读 [OpenAI 的原始 Whisper 论文](https://openai.com/research/whisper)。

## 5.4. 使用 LLM 进行图像描述

我们已经看到，音频可以被当做序列来推理，那么图像呢？您可能认为图像不能被表示为序列，但实际上是可以的！[An Image is Worth 16x16 Words (2020)](https://arxiv.org/abs/2010.11929) 这篇论文讲的就是这个，作者正式提出了视觉 Transformer模型（ViT）。实际上，您可以将原始图像分割成很多块，并通过适当的嵌入策略和创造性的训练方式将这些块视为序列。

> <div><img src="imgs/vit-image-processing.png" width="1000"/></div>
>
> **来源：[Improve Accuracy and Robustness of Vision AI Apps with Vision Transformers and NVIDIA TAO | NVIDIA](https://developer.nvidia.com/blog/improve-accuracy-and-robustness-of-vision-ai-apps-with-vision-transformers-and-nvidia-tao/)**

这就意味着，我们现在有了一个能推理图像的类编码器架构！接下来如何用 ViT 驱动图像描述模型就很明显了，只需再次使用交叉注意力！

实际上，[`nlpconnect/vit-gpt2-image-captioning`](https://huggingface.co/nlpconnect/vit-gpt2-image-captioning) 正是这么做的！它目前是 HuggingFace 上下载量最多的的图像描述模型。模型的架构不会让我们吃惊，只需将上述的视觉编码器与语言解码器组合起来，让它们通过交叉注意力进行交互，再进行训练就可以了！

### 导入 ViT 增强的模型

现在，您应该已经可以轻松的找到需要的 pipeline 了，下面来试试吧！我们导入了一些 [Picasso 项目](https://www.nvidia.com/en-us/gpu-cloud/picasso/)生成的图片。

看看您能不能用 [`nlpconnect/vit-gpt2-image-captioning`](https://huggingface.co/nlpconnect/vit-gpt2-image-captioning) 这类图像描述模型来生成图像的描述。请务必从模型卡了解模型是如何导入的，并验证模型结构是否符合您的预期。总体上，您可以按照预期来使用这个 pipeline。

In [ ]:
import IPython
import PIL

## TODO: Please add some more images into the `img-files` directory
for f in sorted(glob("img-files/*")):
    print(f)
    IPython.display.display(PIL.Image.open(f).resize((300, 200)))

In [ ]:
## TODO: Import the vit pipeline

from transformers import pipeline

vit_pipe = pipeline("image-to-text", model="nlpconnect/vit-gpt2-image-captioning")

In [ ]:
for f in sorted(glob("img-files/*")):
    print(f, vit_pipe(f), '', sep='\n')

您可能会注意到模型的效果实际上并不好。它确实有效，但生成的内容相当简单，格式也不好，还经常识别错对象。

### **任务 1：**拉取更强大的模型

读一下 Salesforce 的 [Bootstrapping Language-Image Pre-training (BLIP) 模型](https://arxiv.org/abs/2201.12086)，并尝试通过 [`Salesforce/blip-image-captioning-large` 模型卡](https://huggingface.co/Salesforce/blip-image-captioning-large)来载入它！BLIP 仍然遵循编码器-解码器的架构，但它有两个编码器层和一些比较特别的预训练目标，因此不太容易手动调整。

> <div><img src="imgs/vit-blip.png" width="1000"/></div>
> 
> **来源：[BLIP: Bootstrapping Language-Image Pre-training for Unified Vision-Language Understanding and Generation](https://paperswithcode.com/paper/blip-bootstrapping-language-image-pre)**

不过还是先把它加载进来，然后重新运行。看看这下结果如何，可以假设 pipeline 会像您预期的那样工作。

In [ ]:
## TODO: Try out the blip model instead
blip_pipe = pipeline("image-to-text", model="Salesforce/blip-image-captioning-large")
for f in sorted(glob("img-files/*")):
    print(f, blip_pipe(f), '', sep='\n')

#### 有条件的图像描述

通过模型卡可以看到，我们可以向图像描述模型添加一些上下文，这样输出就能从我们希望的地方开始。用 pipeline 的话，这个功能不太好实现，但如果直接导入 `BlipProcessor` 和 `BlipForConditionalGeneration` 就会简单很多。查看模型卡，在文本解码器中注入条件，让您的生成内容从“I love cats”开始。看看您是否能得到一些有趣的结果！

In [ ]:
from transformers import BlipProcessor, BlipForConditionalGeneration

## TODO: Import the model components
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")

text_img_pairs = []

for f in sorted(glob("img-files/*")):
    print(f, blip_pipe(f), sep='\n')

    raw_image = PIL.Image.open(f)

    ## TODO: Feel free to change the string. We found it good for first run
    text = "I love cats " 
    # TODO: Perform conditional image captioning and print response
    response = "I don't know"
    
    print(response, '\n')
    text_img_pairs += [(response, raw_image)]
    
## NOTE: The next section depends on this, since text_img_pairs should be aggregated

#### **任务 2：**图像描述

该卡片还提到这个架构可以做图片问答，但没有解释怎么做。作为一项挑战，您可以研究一下怎么在推理过程中指定这些额外信息。或者，在看过 BLIP 的详细信息之后，您可以看看 [NVIDIA 的 NeVA 沙盒](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/playground/collections/neva)，这将让您有机会了解它的工作原理，并推测背后做了些什么不一样的事情。

![NeVA Logo](https://catalog.ngc.nvidia.com/demos/neva.jpg)

稍后我们将提出关于解码器为什么能生成如此合理响应的深刻洞察，不过您可以随时查看 [LLAVA](https://llava-vl.github.io/) 项目，了解使系统如此好用所需的各种技术创新！

In [ ]:
## Room For Code! Feel free to use

## 5.5. 使用 OpenAI CLIP 嵌入

在之前的练习中，您应该从 BLIP 架构中累积了一些文本图像对：

In [ ]:
text_img_pairs

使用这些，我们就可以继续研究使许多语言图像模型更容易训练的主要组件之一：[**CLIP**](https://openai.com/research/clip)，也就是**对比语言图像预训练（Contrastive Language-Image Pre-training）**。

CLIP 提出了一个相当简单的主张：**训练一个语言编码器和一个图像编码器，使得它们能在语言和图像高度相关的时候做出相似的预测**。在大型图像描述数据集上对此任务进行训练后，编码器生成的嵌入确实**既是语义密集的，也是与图像信息一致的**！我们可以快速加载 [HuggingFace CLIP 模型](https://huggingface.co/docs/transformers/model_doc/clip)看看在我们的图像描述池数据上会发生什么：

In [ ]:
import torch
from PIL import Image
import seaborn as sns
import matplotlib.pyplot as plt
from transformers import CLIPProcessor, CLIPModel

# Initialize CLIP model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

texts  = [v[0] for v in text_img_pairs]
images = [v[1] for v in text_img_pairs]

# Process texts and images
inputs_text = processor(text=texts, return_tensors="pt", padding=True)
inputs_images = processor(images=images, return_tensors="pt", padding=True)

# Get embeddings
with torch.no_grad():
    text_embeddings = model.get_text_features(**inputs_text)
    image_embeddings = model.get_image_features(**inputs_images)

# Calculate similarity
similarity = text_embeddings @ image_embeddings.T
similarity = similarity.softmax(dim=1)

# Visualization of the similarity matrix
plt.figure(figsize=(10, 8))
sns.heatmap(similarity.numpy(), annot=True, cmap='coolwarm', xticklabels=texts, yticklabels=texts)
plt.xlabel("Texts")
plt.ylabel("Images")
plt.title("Similarity between Text and Image Embeddings")
plt.show()

可以从它们的 softmax 点积非常清楚地看到，图像的嵌入和基于该图生成的文本的嵌入是很相似的！乍看起来这好像不是特别有用，但实际上对以文本为条件的应用和检索任务来说，它是完美的组件！

* **深度图像先验（Deep Image Priors）：**当您需要为特定问题提供一些图像级语义时，CLIP 是一个很好的起点，因为它已经从预训练任务中建立起了很不错的图像先验。换言之，嵌入器可以自然的作为跨模态的特征提取器！这并不意味着它们可以处理好所有内容，就比如，不能期望它们有涉及 3D 特征的先验，但它们肯定可以用在某些特定的场景下！
* **文本条件（Text-Conditioning）：**CLIP 还可以在经过视觉监督训练的任务中（比如图像生成模型）加上简单的“文本条件”。由于您可以把 CLIP 的图像嵌入用作监督来训练模型，推理的过程就可以提出来使用。简单处理的话效果可能不好，但许多涉及文本和图像模态的论文都会利用 CLIP。
* **检索（Retrieval）：**您可以从矩阵中看到文本和图像的嵌入是一致的。如果图像嵌入存在了数据库中的某个位置，您也许可以利用跟文本嵌入算相似性来进行检索。这样的话，您就可以用文本检索到一些符合预期的图像！

事实上，已经有一个 pipeline 直接利用 CLIP 对图像进行分类。

这就是[**零样本图像分类**](https://huggingface.co/tasks/zero-shot-image-classification)！

<div><img src="imgs/task-zero-shot-img.png" 
     alt="Zero-shot image classification task as seen on https://huggingface.co/tasks/zero-shot-image-classification"
     width="800"/></div>

## 5.6. 总结

之前，我们介绍了用文本作为自然语言生成语境的编码器-解码器架构。在这个 notebook 中，我们展示了相同的逻辑可用于从其它模态生成，比如图像和音频。将语境的模态作为序列并通过交叉注意力机制与解码器交互，就可以轻松的实现了。

**在下一节中，我们将调整方向，找到不使用编码器-解码器结构的理由，继而用更大的模型来追求更通用的语言建模！**

In [ ]:
## Please Run When You're Done!
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)